In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# URLs for datasets
datasets = {
    "SO2TONS": "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/SO2TONS_dataset.csv",
    "NOXTONS": "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/NOXTONS_dataset.csv",
    "COTONS": "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/COTONS_dataset.csv"
}

# Define the days for prediction
specific_dates = {
    "Peak Season": pd.Timestamp("2022-07-15"),
    "Off-Peak Season 1": pd.Timestamp("2022-02-15"),
    "Off-Peak Season 2": pd.Timestamp("2022-10-15"),
}

# Define lakes (sources)
sources = ["LAKE-1", "LAKE-2", "LAKE-3", "LAKE-4"]

# Initialize a dictionary to store models and predictions
predictions = {}

# Loop through each dataset (SO2TONS, NOXTONS, COTONS)
for parameter, url in datasets.items():
    # Load the dataset
    data = pd.read_csv(url)

    # Convert the 'date' column to datetime
    data['date'] = pd.to_datetime(data['date'])

    # Separate data by source
    for source in sources:
        source_data = data[data['Source'] == source]

        # Check if the source data has enough rows
        if source_data.empty or len(source_data) < 10:
            print(f"Not enough data for {parameter} at {source}. Skipping...")
            continue

        # Define predictors and target
        predictors = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres']
        target = 'Emissions_Load'

        # Drop rows with missing values
        source_data = source_data.dropna(subset=predictors + [target])

        # Split into features (X) and target (y)
        X = source_data[predictors]
        y = source_data[target]

        # Split into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train Random Forest Regressor
        rf = RandomForestRegressor(random_state=42, n_estimators=100)
        rf.fit(X_train, y_train)

        # Evaluate the model
        y_pred = rf.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        print(f"Model for {parameter} at {source} (Random Forest):")
        print(f"  RMSE: {rmse:.4f}")
        print(f"  R²: {r2:.4f}")

        # Save predictions for specific days
        predictions[(parameter, source)] = {}
        for season, specific_date in specific_dates.items():
            # Check if the specific date exists in the source data
            day_data = source_data[source_data['date'] == specific_date]
            if not day_data.empty:
                # Extract feature values for the specific day
                specific_features = day_data[predictors]
                specific_actual = day_data[target].iloc[0]

                # Predict emissions/load for the specific day
                specific_prediction = rf.predict(specific_features)[0]

                # Save the prediction and actual value for verification
                predictions[(parameter, source)][season] = {
                    "features": day_data[predictors].iloc[0],
                    "actual": specific_actual,
                    "predicted": specific_prediction
                }
            else:
                print(f"No data available for {parameter} at {source} on {specific_date.date()}.")
                predictions[(parameter, source)][season] = None

# Display all predictions at the end
print("\nFinal Predictions:")
for key, season_results in predictions.items():
    parameter, source = key
    print(f"\n{parameter} at {source}:")
    for season, result in season_results.items():
        if result:
            print(f"  {season}:")
            print(f"    Features: {result['features'].to_dict()}")
            print(f"    Actual Emissions_Load: {result['actual']:.4f}")
            print(f"    Predicted Emissions_Load: {result['predicted']:.4f}")
        else:
            print(f"  {season}: No data available.")


Model for SO2TONS at LAKE-1 (Random Forest):
  RMSE: 0.0000
  R²: -0.1794
No data available for SO2TONS at LAKE-1 on 2022-10-15.
Model for SO2TONS at LAKE-2 (Random Forest):
  RMSE: 0.0000
  R²: -0.0291
No data available for SO2TONS at LAKE-2 on 2022-02-15.
Model for SO2TONS at LAKE-3 (Random Forest):
  RMSE: 0.0000
  R²: -0.2116
Model for SO2TONS at LAKE-4 (Random Forest):
  RMSE: 0.0000
  R²: 0.0730
Model for NOXTONS at LAKE-1 (Random Forest):
  RMSE: 0.0003
  R²: -0.1219
No data available for NOXTONS at LAKE-1 on 2022-10-15.
Model for NOXTONS at LAKE-2 (Random Forest):
  RMSE: 0.0003
  R²: -0.2171
No data available for NOXTONS at LAKE-2 on 2022-02-15.
Model for NOXTONS at LAKE-3 (Random Forest):
  RMSE: 0.0003
  R²: -0.0591
Model for NOXTONS at LAKE-4 (Random Forest):
  RMSE: 0.0003
  R²: 0.0044
Model for COTONS at LAKE-1 (Random Forest):
  RMSE: 0.0002
  R²: -0.3069
No data available for COTONS at LAKE-1 on 2022-10-15.
Model for COTONS at LAKE-2 (Random Forest):
  RMSE: 0.0003
  R²